In [1]:
### Helper function for uploading MNIST dataset ###

import os
import struct
import random
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
%matplotlib inline

"""
Loosely inspired by http://abel.ee.ucla.edu/cvxopt/_downloads/mnist.py
which is GPL licensed.
"""

def read(dataset = "training", path = "."):
    """
    Python function for importing the MNIST data set.  It returns an iterator
    of 2-tuples with the first element being the label and the second element
    being a numpy.uint8 2D array of pixel data for the given image.
    """

    if dataset is "training":
        fname_img = os.path.join(path, 'train-images-idx3-ubyte/data')
        fname_lbl = os.path.join(path, 'train-labels-idx1-ubyte/data')
    elif dataset is "testing":
        fname_img = os.path.join(path, 't10k-images-idx3-ubyte/data')
        fname_lbl = os.path.join(path, 't10k-labels-idx1-ubyte/data')
    else:
        raise ValueError("dataset must be 'testing' or 'training'")

    # Load everything in some numpy arrays
    with open(fname_lbl, 'rb') as flbl:
        magic, num = struct.unpack(">II", flbl.read(8))
        lbl = np.fromfile(flbl, dtype=np.int8)

    with open(fname_img, 'rb') as fimg:
        magic, num, rows, cols = struct.unpack(">IIII", fimg.read(16))
        img = np.fromfile(fimg, dtype=np.uint8).reshape(len(lbl), rows, cols)

    get_img = lambda idx: (lbl[idx], img[idx])

    # Create an iterator which returns each image in turn
    for i in range(len(lbl)):
        yield get_img(i)

In [2]:
### Helper function turning MNIST data to matrix ###
def toMatrix(data, maxItems=1000):
    datalist = [t for t in data]
    m = maxItems
    n = 28 * 28
    X = np.matrix(np.zeros(m * n)).reshape(m, n)
    Y = np.matrix(np.zeros(m)).reshape(m, 1)
    for i, (label, image) in enumerate(datalist[:m]):
        X[i] = image.reshape(28*28,)
        Y[i] = label
    return X, Y

In [3]:
def normalize_grayscale(image_data):
    """
    Normalize the image data with Min-Max scaling to a range of [0.1, 0.9]
    :param image_data: The image data to be normalized
    :return: Normalized image data
    """
    a = -1.0
    b = 1.0
    grayscale_min = 0
    grayscale_max = 255
    return a + ( ( (image_data - grayscale_min)*(b - a) )/( grayscale_max - grayscale_min ) )

In [4]:
### Reading training and testing data ###

data_train = list(read("training"))
data_test = list(read("testing"))

train = toMatrix(data_train)
test = toMatrix(data_test)

#train_features = train[0]
train_features = normalize_grayscale(train[0])
train_labels = train[1].astype(int)

test_features = test[0]
test_labels = test[1].astype(int)

In [5]:
#print((0.1+(test_features-0)*(0.9-0.1)/(255-0))[0])
#print(train_features[0])

In [6]:
n_features = 784
n_labels = 10
weights = tf.Variable(tf.truncated_normal((n_features, n_labels)))

In [7]:
bias = tf.Variable(tf.zeros(n_labels))

In [8]:
def linear_prediction(features, weights, bias):
    return tf.add(tf.matmul(features, weights), bias)

In [9]:
learning_rate = 0.5
epochs = 5
batch_size = 100

features = tf.placeholder(tf.float32)
labels = tf.placeholder(tf.int32)

logits = linear_prediction(features, weights, bias)
one_hot = tf.one_hot(labels, n_labels)

scores = tf.nn.softmax(logits)
cross = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=one_hot, logits=logits))
loss = tf.reduce_mean(cross)

optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

In [12]:
output = None
import math
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    batch_count = int(math.ceil(len(train_features)/batch_size))
    
    for epoch_i in range(epochs):
        
        for batch_i in range(batch_count):
            batch_start = batch_i*batch_size
            batch_features = train_features[batch_start:batch_start + batch_size]
            batch_labels = train_labels[batch_start:batch_start + batch_size]

            _, l = sess.run([optimizer, loss], feed_dict={features: batch_features, labels: batch_labels})
    
    scores, l = sess.run([scores, loss], feed_dict={features: train_features, labels: train_labels})
#     _, l = sess.run([optimizer, loss], feed_dict={features: train_features, labels: train_labels})
    output = sess.run(scores, feed_dict={features: train_features, labels: train_labels})
    
    test_score = sess.run(scores, feed_dict={features: test_features, labels: test_labels})
    

TypeError: Fetch argument array([[  1.76102457e-08,   3.28232511e-11,   2.56745980e-09, ...,
          1.62512578e-15,   1.91084987e-18,   8.69796077e-06],
       [  1.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          3.95612888e-32,   3.60997658e-33,   7.75110997e-23],
       [  1.09388118e-17,   1.55346227e-24,   4.80458133e-20, ...,
          1.26742112e-15,   1.23265394e-26,   3.88975296e-10],
       ..., 
       [  1.00000000e+00,   2.63688385e-28,   1.85968732e-20, ...,
          2.42714948e-25,   4.16661684e-31,   8.67049224e-26],
       [  8.33012565e-12,   3.87392085e-15,   2.30162081e-13, ...,
          8.14120889e-01,   1.54789416e-19,   3.21158849e-18],
       [  3.25286682e-19,   4.96809015e-14,   2.06380438e-17, ...,
          4.58500074e-20,   2.14207205e-27,   1.00280389e-15]], dtype=float32) has invalid type <class 'numpy.ndarray'>, must be a string or Tensor. (Can not convert a ndarray into a Tensor or Operation.)

In [10]:
verbose = 0 # set this to 1 if you want to see actual classifications
accuracy = 0.0
for i in range(len(train_features)):
    pred = np.argmax(output[i])
    actual = train_labels[i, 0]
    if verbose == 1:
        print(str(i+1) + ". Predicted:", pred, "Label:", actual, "Predictions:", np.round(output[i],4))
    accuracy += 1 if pred == actual else 0
print("Accuracy = " + str(accuracy/len(train_features)))

Accuracy = 0.754
